## 下載 anything-llm 進行編譯

In [22]:
%%bash
rm -rf anything-llm
git clone https://github.com/Mintplex-Labs/anything-llm.git

Cloning into 'anything-llm'...


In [23]:
%%bash
cat << \EOF >  demo.script
DIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" >/dev/null 2>&1 && pwd )"

echo "fix"
sed -i "s/5.3.0/5.3.1/g" ${DIR}/anything-llm//server/package.json

echo "Rebuilding Frontend"
cd ${DIR}/anything-llm/frontend && yarn && yarn build

echo "Copying to Sever Public"
rm -rf ${DIR}/anything-llm/server/public
cp -r ${DIR}/anything-llm/frontend/dist ${DIR}/anything-llm/server/public

echo "Installing collector dependencies"
cd ${DIR}/anything-llm/collector && yarn

echo "Installing server dependencies & running migrations"
cd ${DIR}/anything-llm/server && yarn
cd ${DIR}/anything-llm/server && npx prisma migrate deploy --schema=./prisma/schema.prisma
cd ${DIR}/anything-llm/server && npx prisma generate

echo ".env"
touch ${DIR}/anything-llm/server/.env

EOF

chmod 755 demo.script

In [24]:
%%bash
ml libs/singularity/3.10.2
singularity exec -B /work /work/u00cjz00/nvidia/node_hydrogen-bullseye.sif ./demo.script

/home/g00cjz00/demo2024/llm_course_20231228/demo.script: 1: Bad substitution


fix
Rebuilding Frontend
yarn install v1.22.19
[1/4] Resolving packages...
[2/4] Fetching packages...
[3/4] Linking dependencies...


warning " > @esbuild-plugins/node-globals-polyfill@0.1.1" has unmet peer dependency "esbuild@*".


[4/4] Building fresh packages...
Done in 7.19s.
yarn run v1.22.19
$ vite build
vite v4.5.0 building for production...
transforming...
✓ 3163 modules transformed.
rendering chunks...
computing gzip size...
dist/assets/slash-commands-icon-16b44cbd.svg      0.35 kB │ gzip:   0.22 kB
dist/index.html                                   1.69 kB │ gzip:   0.55 kB
dist/assets/milvus-859aae61.png                   4.25 kB
dist/assets/mistral-7617df59.jpeg                 4.54 kB
dist/assets/youtube-b0678cf5.png                  5.41 kB
dist/assets/anything-llm-67ade0a8.png             6.32 kB
dist/assets/togetherai-23bbd12b.png               8.36 kB
dist/assets/anthropic-f5b83291.png               11.89 kB
dist/assets/zilliz-3dd28de0.png                  14.34 kB
dist/assets/qdrant-e1811442.png                  15.07 kB
dist/assets/huggingface-e6e5045d.png             17.57 kB
dist/assets/create-workspace-7b54259b.png        18.41 kB
dist/assets/github-4d0d04cb.png                  22.06 kB
dist/


(!) Some chunks are larger than 500 kBs after minification. Consider:
- Using dynamic import() to code-split the application
- Use build.rollupOptions.output.manualChunks to improve chunking: https://rollupjs.org/configuration-options/#output-manualchunks
- Adjust chunk size limit for this warning via build.chunkSizeWarningLimit.


...

[4/5] Linking dependencies...


warning " > @qdrant/js-client-rest@1.4.0" has unmet peer dependency "typescript@>=4.1".


[5/5] Building fresh packages...


warning @datastax/astra-db-ts@0.1.3: Unable to read "bin" directory of module "@datastax/astra-db-ts"


success Saved lockfile.
Done in 11.71s.
Prisma schema loaded from prisma/schema.prisma
Datasource "db": SQLite database "anythingllm.db" at "file:../storage/anythingllm.db"

SQLite database anythingllm.db created at file:../storage/anythingllm.db

11 migrations found in prisma/migrations

Applying migration `20230921191814_init`
Applying migration `20231101001441_init`
Applying migration `20231101195421_init`
Applying migration `20231129012019_add`
Applying migration `20240113013409_init`
Applying migration `20240118201333_init`
Applying migration `20240202002020_init`
Applying migration `20240206181106_init`
Applying migration `20240206211916_init`
Applying migration `20240208224848_init`
Applying migration `20240210004405_init`

The following migrations have been applied:

migrations/
  └─ 20230921191814_init/
    └─ migration.sql
  └─ 20231101001441_init/
    └─ migration.sql
  └─ 20231101195421_init/
    └─ migration.sql
  └─ 20231129012019_add/
    └─ migration.sql
  └─ 2024011301

## 啟動  anything-llm

In [28]:
%%bash
ml libs/singularity/3.10.2
export SINGULARITYENV_STORAGE_DIR="/app/server/storage"
nohup singularity run -B anything-llm:/app /work/u00cjz00/nvidia/anythingllm_latest.sif > ./anything-llm.log 2>&1 &

In [32]:
!/usr/sbin/lsof -i -P -n |grep node

node       62388 g00cjz00   18u  IPv6 301598461      0t0  TCP *:8888 (LISTEN)
node       62534 g00cjz00   19u  IPv6 301550153      0t0  TCP *:3001 (LISTEN)


## 連線

In [33]:
%%bash
node_hostname=$(hostname -s)
node_ip=$(cat /etc/hosts |grep "$(hostname -a)" | awk '{print $1}')
# PORT
noed_port_genai=$(python -c "import socket; s = socket.socket(socket.AF_INET, socket.SOCK_STREAM); s.bind(('', 0)); addr = s.getsockname(); s.close(); print(addr[1])")
noed_port_genai=3001

# PROXY
proxy_url=/rstudio/${node_ip}/${noed_port_genai}
# URL
https_url=https://node01.biobank.org.tw${proxy_url}/

# SSH FORWARDING
ssh_cmd="ssh -o StrictHostKeyChecking=no -o TCPKeepAlive=yes -o ServerAliveCountMax=20 -o ServerAliveInterval=15 -NfL ${noed_port_genai}:${node_hostname}:${noed_port_genai} $(whoami)@t3-c4.nchc.org.tw"; 
echo "SSH:"
echo ${ssh_cmd}
echo ""
ssh_cmd="ssh -L ${noed_port_genai}:${node_hostname}:${noed_port_genai} $(whoami)@t3-c4.nchc.org.tw"
echo "SSH:"
echo ${ssh_cmd}
echo ""
echo "URL: http://localhost:${noed_port_genai}"
echo ""

SSH:
ssh -o StrictHostKeyChecking=no -o TCPKeepAlive=yes -o ServerAliveCountMax=20 -o ServerAliveInterval=15 -NfL 3001:gpn3002:3001 g00cjz00@t3-c4.nchc.org.tw

SSH:
ssh -L 3001:gpn3002:3001 g00cjz00@t3-c4.nchc.org.tw

URL: http://localhost:3001



## DELETE JOB

In [34]:
!ps -ef |grep node | awk '{print $2}' | xargs kill -9

kill: sending signal to 7340 failed: Operation not permitted
kill: sending signal to 8598 failed: Operation not permitted
kill: sending signal to 64189 failed: No such process
